<a href="https://colab.research.google.com/github/Akshaya-23-27/RAG-/blob/main/RAG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 2 chat with PDF using RAG pipeline

Below is a Python implementation of a Retrieval-Augmented Generation (RAG) pipeline for interacting with websites. The pipeline uses libraries like BeautifulSoup for web scraping, sentence-transformers for embeddings, FAISS for the vector database, and OpenAI for response generation.

Implementaion

In [2]:

def crawl_and_scrape(url):
    """
    Crawl and scrape content from a website.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract textual content from <p> and <h> tags
    paragraphs = soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"])
    content = [p.get_text().strip() for p in paragraphs if p.get_text().strip()]
    return content

def segment_and_embed(content, chunk_size=100):
    """
    Segment content into smaller chunks and compute embeddings.
    """
    chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
    embeddings = embedding_model.encode([" ".join(chunk) for chunk in chunks])
    return chunks, embeddings

def store_embeddings(chunks, embeddings, url):
    """
    Store embeddings in the vector database with metadata.
    """
    global metadata_store
    index.add(embeddings)
    for i, chunk in enumerate(chunks):
        metadata_store.append({"url": url, "chunk": " ".join(chunk)})

def query_database(user_query, top_k=5):
    """
    Query the vector database and retrieve relevant chunks.
    """
    query_embedding = embedding_model.encode([user_query])
    distances, indices = index.search(query_embedding, top_k)
    results = [metadata_store[idx] for idx in indices[0] if idx < len(metadata_store)]
    return results

def generate_response(retrieved_data, user_query):
    """
    Generate a response using OpenAI GPT with retrieved data.
    """
    context = "\n\n".join([f"Chunk from {item['url']}:\n{item['chunk']}" for item in retrieved_data])
    prompt = f"""
    You are an AI assistant with access to the following information:
    {context}

    User's query: {user_query}
    Provide a detailed and accurate response using the above context.
    """
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=500,
        temperature=0
    )
    return response["choices"][0]["text"].strip()



Key Steps Explained

1.Data Ingestion:

Crawls and scrapes websites using requests and BeautifulSoup. Segments textual data into smaller chunks for granularity. Embedding and Storage:

Computes embeddings for each chunk using a SentenceTransformer model. Stores embeddings in FAISS for efficient similarity-based retrieval.

2.Query Handling:

Embeds the user query and performs a similarity search in the FAISS index. Retrieves the top-k relevant chunks based on similarity scores. Response Generation:

Passes retrieved chunks to OpenAI GPT along with the user's query to generate a context-rich response.

Task 1 : chat with website using RAG pipeline

To implement a Retrieval-Augmented Generation (RAG) pipeline for chatting with PDFs, you can use the following stack:

PDF parsing and chunking: PyPDF2 or pdfplumber for extracting text and logical segmentation.
Embeddings: sentence-transformers for generating vector embeddings.
Vector database: FAISS, Weaviate, or Pinecone for efficient similarity-based retrieval.
LLM integration: OpenAI API or Hugging Face for generating responses.
Framework: LangChain or a custom framework to integrate all components.

In [5]:
# Step 1: Data Ingestion
def ingest_pdfs(pdf_files):
    """Extract and chunk text from PDFs, and store embeddings in a vector database."""
    docs = []
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        docs.append(text)

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = []
    for doc in docs:
        chunks.extend(text_splitter.split_text(doc))

    # Generate embeddings and store in a vector database
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embeddings)

    return vector_store

# Step 2: Query Handling
def handle_query(vector_store, query, llm_model="gpt-3.5-turbo"):
    """Handle user queries by retrieving relevant data and generating a response."""
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA(llm=OpenAI(model=llm_model), retriever=retriever)

    response = qa_chain.run(query)
    return response

# Step 3: Comparison Queries
def handle_comparison_query(vector_store, comparison_query):
    """Perform comparison queries and generate a structured response."""
    # Retrieve relevant chunks
    retriever = vector_store.as_retriever()
    docs = retriever.get_relevant_documents(comparison_query)

    # Aggregate and process data
    comparison_data = {}
    for doc in docs:
        # Add logic to parse fields for comparison
        # For simplicity, we'll assume each document contains structured data
        comparison_data[doc.metadata["source"]] = doc.page_content

    # Generate a structured response (example: tabular format)
    response = "Comparison:\n"
    for source, content in comparison_data.items():
        response += f"- Source: {source}\n  Content: {content}\n"

    return response

Explanation of Components

1.Data Ingestion:

. PDFs are parsed using PyPDF2
. Text is chunked into manageable parts using
 RecursiveCharacterTextSplitter.
. Vector embeddings are generated using a pre-trained embedding model (sentence-transformers).

2.Query Handling:

User queries are embedded and matched with the stored embeddings in the vector database.
Relevant chunks are retrieved, and the LLM generates responses.

3.Comparison Queries:

Specific terms or fields for comparison are identified.
Retrieved chunks are processed to generate structured comparisons.

4.Response Generation:

The LLM produces detailed and natural-language responses using retrieval-augmented context.

Set up OpenAI API key :

In [8]:
openai.api_key = "YOUR_OPENAI_API_KEY"